### import the dependencies

In [5]:
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.utils.class_weight import compute_class_weight
from imblearn.over_sampling import SMOTE

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader

from torchdiffeq import odeint

from sklearn.metrics import accuracy_score, f1_score, classification_report, confusion_matrix

RANDOM_SEED = 42
torch.manual_seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)

DATA_PATH = os.path.join("..", "Dataset", "raw_RRI_segments.csv")
OUT_MODEL = os.path.join("..", "Three_Class_Models/saved_models", "NODE_PSR_best.pth")

### Load PSR dataset and preprocess

In [6]:
df = pd.read_csv(DATA_PATH)
# pick RRI feature columns (prefix 'r_')
feature_cols = [c for c in df.columns if c.startswith("r_")]
meta_cols = ["patient_id", "record_id", "label", "label_str", "orig_len"]

assert len(feature_cols) > 0, "No r_ columns found in CSV"


In [7]:
def phase_space_reconstruct(x, m=3, tau=1):
    """
    x: 1D array of RRI
    m: embedding dimension
    tau: time delay
    Returns flattened PSR embedding: x(t), x(t+tau), ..., x(t+(m-1)*tau)
    """
    x = np.asarray(x)
    N = len(x)
    if N < (m-1)*tau + 1:
        # pad with zeros if too short
        x = np.pad(x, (0, (m-1)*tau + 1 - N), 'constant')
        N = len(x)
    psr_vectors = [x[i:N-(m-1)*tau + i] for i in range(m)]
    psr_flat = np.column_stack(psr_vectors).flatten()
    return psr_flat

# Pick RRI columns
rri_cols = [f"r_{i}" for i in range(50)]

# Compute PSR for all rows with m=3, tau=2
psr_features = df[rri_cols].apply(lambda row: phase_space_reconstruct(row.values, m=3, tau=2), axis=1)
psr_features = np.stack(psr_features.values)  # shape: (num_samples, m*(N-(m-1)*tau))

# Add PSR columns to DataFrame
num_psr_cols = psr_features.shape[1]
psr_col_names = [f"psr_{i}" for i in range(num_psr_cols)]
df_psr = pd.DataFrame(psr_features, columns=psr_col_names, index=df.index)
df = pd.concat([df, df_psr], axis=1)

print("PSR features added (m=3, tau=2). DataFrame shape:", df.shape)




PSR features added (m=3, tau=2). DataFrame shape: (14205, 193)


In [8]:
feature_cols = psr_col_names
meta_cols = ["patient_id", "record_id", "label", "label_str", "orig_len"]

# Feature matrix X
X = df[feature_cols].values.astype(np.float32)

# Keep label as float first (to handle NaN/inf safely)
y_raw = df["label"].values

# Step 1: Drop rows with invalid X (NaN or inf)
mask_good = np.isfinite(X).all(axis=1)

# Step 2: Drop invalid labels (NaN, inf, negative)
mask_good &= np.isfinite(y_raw) & (y_raw >= 0)

# Apply combined mask
X = X[mask_good]
y = y_raw[mask_good].astype(int)

# Step 3: Inspect
print(np.unique(y)[:10])   # print first few unique labels
print(y[:20])

# Step 4: Safe bincount
print("Loaded samples:", X.shape, "class counts:", np.bincount(y))


[0 1 2]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
Loaded samples: (14204, 138) class counts: [4750 4707 4747]


### Train / val / test split, scaling and SMOTE (apply SMOTE only to training set)

In [9]:
X_temp, X_test, y_temp, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=RANDOM_SEED
)

X_train, X_val, y_train, y_val = train_test_split(
    X_temp, y_temp, test_size=0.2, stratify=y_temp, random_state=RANDOM_SEED
)

print("Splits -> train:", X_train.shape, "val:", X_val.shape, "test:", X_test.shape)

# Standardize using training stats
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)
X_test = scaler.transform(X_test)

# SMOTE on training only
smote = SMOTE(random_state=RANDOM_SEED)
X_train_res, y_train_res = smote.fit_resample(X_train, y_train)
print("After SMOTE train distribution:", np.bincount(y_train_res))

Splits -> train: (9090, 138) val: (2273, 138) test: (2841, 138)
After SMOTE train distribution: [3040 3040 3040]


### Compute class weights

In [10]:
# compute class weights from resampled training set (used as alpha for focal loss)
cw = compute_class_weight(class_weight='balanced', classes=np.unique(y_train_res), y=y_train_res)
class_weights_tensor = torch.tensor(cw, dtype=torch.float32)
print("Class weights:", cw)

Class weights: [1. 1. 1.]


### Compute focal loss

In [11]:
class FocalLoss(nn.Module):
    def __init__(self, alpha=None, gamma=2.0):
        super(FocalLoss, self).__init__()
        self.alpha = alpha
        self.gamma = gamma

    def forward(self, inputs, targets):
        ce = F.cross_entropy(inputs, targets, reduction='none', weight=self.alpha)
        pt = torch.exp(-ce)
        loss = ((1 - pt) ** self.gamma) * ce
        return loss.mean()

### Define model class

In [12]:
class ODEFunc(nn.Module):
    def __init__(self, dim):
        super(ODEFunc, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(dim, 64),
            nn.Tanh(),
            nn.Linear(64, dim)
        )

    def forward(self, t, x):
        return self.net(x)

class NODEModel(nn.Module):
    def __init__(self, dim, num_classes):
        super(NODEModel, self).__init__()
        self.odefunc = ODEFunc(dim)
        self.classifier = nn.Sequential(
            nn.Linear(dim, 64),
            nn.ReLU(),
            nn.Linear(64, num_classes)
        )

    def forward(self, x):
        # x: (batch, features) — treat as initial state
        t = torch.tensor([0.0, 1.0], dtype=x.dtype, device=x.device)
        out = odeint(self.odefunc, x, t)[-1]
        return self.classifier(out)

### Train model

In [13]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

In [14]:
# --- Simple training function ---
def quick_train(model_class, input_dim, num_classes,
                train_dataset, val_dataset,
                class_weights_tensor,
                lr=1e-3,
                batch_size=32,
                epochs=5,
                device="cuda",
                save_path=None):

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

    model = model_class(dim=input_dim, num_classes=num_classes).to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    criterion = FocalLoss(alpha=class_weights_tensor.to(device), gamma=2.0)

    best_f1 = 0.0
    best_state = None

    for epoch in range(1, epochs + 1):
        model.train()
        running_loss = 0.0
        for xb, yb in train_loader:
            xb, yb = xb.to(device), yb.to(device)
            optimizer.zero_grad()
            out = model(xb)
            loss = criterion(out, yb)
            loss.backward()
            optimizer.step()
            running_loss += loss.item() * xb.size(0)
        train_loss = running_loss / len(train_loader.dataset)

        # --- Validation ---
        model.eval()
        preds, trues = [], []
        with torch.no_grad():
            for xb, yb in val_loader:
                xb, yb = xb.to(device), yb.to(device)
                out = model(xb)
                _, p = torch.max(out, 1)
                preds.extend(p.cpu().numpy())
                trues.extend(yb.cpu().numpy())

        val_f1 = f1_score(trues, preds, average="weighted")
        print(f"Epoch {epoch}: train_loss={train_loss:.4f}, val_f1={val_f1:.4f}")

        if val_f1 > best_f1:
            best_f1 = val_f1
            best_state = model.state_dict().copy()
            if save_path:
                torch.save(best_state, save_path)
                print("  -> New best model saved.")

    print(f"\n✅ Training done. Best val F1 = {best_f1:.4f}")
    return model, best_f1, best_state

In [15]:
train_dataset = TensorDataset(
    torch.from_numpy(X_train_res.astype(np.float32)),
    torch.from_numpy(y_train_res.astype(np.int64))
)
val_dataset = TensorDataset(
    torch.from_numpy(X_val.astype(np.float32)),
    torch.from_numpy(y_val.astype(np.int64))
)

input_dim = X_train_res.shape[1]
num_classes = int(np.unique(y).size)

model, best_f1, best_state = quick_train(
    model_class=NODEModel,
    input_dim=input_dim,
    num_classes=num_classes,
    train_dataset=train_dataset,
    val_dataset=val_dataset,
    class_weights_tensor=class_weights_tensor,
    lr=1e-3,
    batch_size=32,
    epochs=5,
    device=device,
    save_path=OUT_MODEL
)

Epoch 1: train_loss=0.3230, val_f1=0.5762
  -> New best model saved.
Epoch 2: train_loss=0.3032, val_f1=0.5890
  -> New best model saved.
Epoch 3: train_loss=0.2916, val_f1=0.6053
  -> New best model saved.
Epoch 4: train_loss=0.2815, val_f1=0.6150
  -> New best model saved.
Epoch 5: train_loss=0.2717, val_f1=0.6030

✅ Training done. Best val F1 = 0.6150


In [16]:
# --- Evaluate on validation set and print classification report ---
from sklearn.metrics import classification_report

model.eval()
val_preds, val_trues = [], []

with torch.no_grad():
    for xb, yb in DataLoader(val_dataset, batch_size=32, shuffle=False):
        xb, yb = xb.to(device), yb.to(device)
        outputs = model(xb)
        preds = outputs.argmax(dim=1)
        val_preds.extend(preds.cpu().numpy())
        val_trues.extend(yb.cpu().numpy())

print("\nClassification Report (Validation):")
print(classification_report(val_trues, val_preds, digits=4))



Classification Report (Validation):
              precision    recall  f1-score   support

           0     0.5128    0.6053    0.5552       760
           1     0.5492    0.3559    0.4319       753
           2     0.7613    0.8895    0.8204       760

    accuracy                         0.6177      2273
   macro avg     0.6078    0.6169    0.6025      2273
weighted avg     0.6079    0.6177    0.6030      2273



In [17]:
import itertools
from sklearn.metrics import f1_score

def train_with_hyperparameter_tuning(model_class, input_dim, num_classes,
                                     train_dataset, val_dataset,
                                     class_weights_tensor,
                                     param_grid,
                                     save_path=OUT_MODEL,
                                     epochs=8,
                                     patience=2,
                                     device=device):
    class_weights_tensor = class_weights_tensor.to(device)

    best_f1 = 0.0
    best_params = None
    best_state = None

    combos = list(itertools.product(*param_grid.values()))
    print(f"Total combinations: {len(combos)}")

    for combo in combos:
        params = dict(zip(param_grid.keys(), combo))
        print("\nTrying params:", params)

        batch_size = params["batch_size"]
        train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
        val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

        model = model_class(dim=input_dim, num_classes=num_classes).to(device)
        if params["optimizer"] == "adam":
            optimizer = torch.optim.Adam(model.parameters(), lr=params["lr"])
        elif params["optimizer"] == "sgd":
            optimizer = torch.optim.SGD(model.parameters(), lr=params["lr"], momentum=0.9)
        else:
            raise ValueError("Unsupported optimizer")

        criterion = FocalLoss(alpha=class_weights_tensor, gamma=params.get("gamma", 2.0))
        epochs_no_improve = 0
        local_best = 0.0

        for epoch in range(1, epochs + 1):
            model.train()
            running_loss = 0.0
            for xb, yb in train_loader:
                xb, yb = xb.to(device), yb.to(device)
                optimizer.zero_grad()
                out = model(xb)
                loss = criterion(out, yb)
                loss.backward()
                optimizer.step()
                running_loss += loss.item() * xb.size(0)
            train_loss = running_loss / len(train_loader.dataset)

            model.eval()
            preds, trues = [], []
            with torch.no_grad():
                for xb, yb in val_loader:
                    xb, yb = xb.to(device), yb.to(device)
                    out = model(xb)
                    _, p = torch.max(out, dim=1)
                    preds.extend(p.cpu().numpy()); trues.extend(yb.cpu().numpy())

            val_f1 = f1_score(trues, preds, average="weighted")
            print(f"  epoch {epoch} train_loss: {train_loss:.4f} val_f1: {val_f1:.4f}")

            if val_f1 > local_best:
                local_best = val_f1
                epochs_no_improve = 0
            else:
                epochs_no_improve += 1
                if epochs_no_improve >= patience:
                    print("  early stopping")
                    break

        # update global best
        if local_best > best_f1:
            best_f1 = local_best
            best_params = params
            best_state = model.state_dict().copy()
            torch.save(best_state, save_path)
            print("  -> New global best. Saved model.")

    print(f"\nGlobal best val F1: {best_f1:.4f} params: {best_params}")
    return best_params, best_f1, best_state

# Build datasets (reuse existing X_train_res, y_train_res, X_val, y_val)
train_dataset = TensorDataset(torch.from_numpy(X_train_res.astype(np.float32)), torch.from_numpy(y_train_res.astype(np.int64)))
val_dataset = TensorDataset(torch.from_numpy(X_val.astype(np.float32)), torch.from_numpy(y_val.astype(np.int64)))
test_dataset = TensorDataset(torch.from_numpy(X_test.astype(np.float32)), torch.from_numpy(y_test.astype(np.int64)))

input_dim = X_train_res.shape[1]
num_classes = int(np.unique(y).size)

param_grid = {
    "lr": [1e-3, 5e-4],
    "batch_size": [16, 32],
    "optimizer": ["adam", "sgd"]
}

best_params, best_f1, best_state = train_with_hyperparameter_tuning(
    model_class=NODEModel,
    input_dim=input_dim,
    num_classes=num_classes,
    train_dataset=train_dataset,
    val_dataset=val_dataset,
    class_weights_tensor=class_weights_tensor,
    param_grid=param_grid,
    save_path=OUT_MODEL,
    epochs=8,
    patience=2,
    device=device
)

print("Best params:", best_params, "Best val F1:", best_f1)


Total combinations: 8

Trying params: {'lr': 0.001, 'batch_size': 16, 'optimizer': 'adam'}
  epoch 1 train_loss: 0.3237 val_f1: 0.6025
  epoch 2 train_loss: 0.3024 val_f1: 0.5817
  epoch 3 train_loss: 0.2885 val_f1: 0.6180
  epoch 4 train_loss: 0.2793 val_f1: 0.5932
  epoch 5 train_loss: 0.2653 val_f1: 0.5766
  early stopping
  -> New global best. Saved model.

Trying params: {'lr': 0.001, 'batch_size': 16, 'optimizer': 'sgd'}
  epoch 1 train_loss: 0.3349 val_f1: 0.5813
  epoch 2 train_loss: 0.3156 val_f1: 0.5805
  epoch 3 train_loss: 0.3101 val_f1: 0.5947
  epoch 4 train_loss: 0.3059 val_f1: 0.5824
  epoch 5 train_loss: 0.3022 val_f1: 0.5702
  early stopping

Trying params: {'lr': 0.001, 'batch_size': 32, 'optimizer': 'adam'}
  epoch 1 train_loss: 0.3219 val_f1: 0.5840
  epoch 2 train_loss: 0.3012 val_f1: 0.6160
  epoch 3 train_loss: 0.2882 val_f1: 0.5941
  epoch 4 train_loss: 0.2786 val_f1: 0.5986
  early stopping

Trying params: {'lr': 0.001, 'batch_size': 32, 'optimizer': 'sgd'}
  

### Final evaluation on test set

In [18]:
# Load best model for final evaluation
if best_state is not None:
    best_model = NODEModel(dim=input_dim, num_classes=num_classes)
    best_model.load_state_dict(best_state)
    best_model = best_model.to(device)
else:
    # fallback: instantiate a fresh model
    best_model = NODEModel(dim=input_dim, num_classes=num_classes).to(device)

# ensure test_loader exists (create if needed)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Final evaluation
best_model.eval()
test_preds, test_trues = [], []
with torch.no_grad():
    for xb, yb in test_loader:
        xb = xb.to(device)
        out = best_model(xb)
        _, p = torch.max(out, dim=1)
        test_preds.extend(p.cpu().numpy())
        test_trues.extend(yb.numpy())

print("Test class counts:", np.bincount(test_trues))
print("Accuracy:", accuracy_score(test_trues, test_preds))
print("F1 (weighted):", f1_score(test_trues, test_preds, average="weighted"))
print("\nClassification report:\n", classification_report(test_trues, test_preds))
print("\nConfusion matrix:\n", confusion_matrix(test_trues, test_preds))

Test class counts: [950 941 950]
Accuracy: 0.6300598380851813
F1 (weighted): 0.6314562385528016

Classification report:
               precision    recall  f1-score   support

           0       0.52      0.51      0.52       950
           1       0.54      0.57      0.55       941
           2       0.84      0.81      0.82       950

    accuracy                           0.63      2841
   macro avg       0.63      0.63      0.63      2841
weighted avg       0.63      0.63      0.63      2841


Confusion matrix:
 [[486 380  84]
 [341 535  65]
 [104  77 769]]
